In [1]:
import tensorflow as tf
import string
import requests
import pandas as pd

In [2]:
response = requests.get('https://raw.githubusercontent.com/laxmimerit/poetry-data/master/adele.txt')

In [3]:
response

<Response [200]>

In [4]:
response.text

'Looking for some education\nMade my way into the night\nAll that bullshit conversation\nBaby, can\'t you read the signs? I won\'t bore you with the details, baby\nI don\'t even wanna waste your time\nLet\'s just say that maybe\nYou could help me ease my mind\nI ain\'t Mr. Right But if you\'re looking for fast love\nIf that\'s love in your eyes\nIt\'s more than enough\nHad some bad love\nSo fast love is all that I\'ve got on my mind Ooh, ooh\nOoh, ooh Looking for some affirmation\nMade my way into the sun\nMy friends got their ladies\nAnd they\'re all having babies\nI just wanna have some fun I won\'t bore you with the details, baby\nI don\'t even wanna waste your time\nLet\'s just say that maybe\nYou could help me ease my mind\nI ain\'t Mr. Right But if you\'re looking for fast love\nIf that\'s love in your eyes\nIt\'s more than enough\nI\'ve had some bad love\nSo fast love is all that I\'ve got on my mind Ooh, ooh\nBaby, baby\nOoh, ooh\nBaby, baby\nOoh, ooh\nBaby, baby\nOoh In the ab

In [5]:
data=response.text.splitlines()
print(data)

['Looking for some education', 'Made my way into the night', 'All that bullshit conversation', "Baby, can't you read the signs? I won't bore you with the details, baby", "I don't even wanna waste your time", "Let's just say that maybe", 'You could help me ease my mind', "I ain't Mr. Right But if you're looking for fast love", "If that's love in your eyes", "It's more than enough", 'Had some bad love', "So fast love is all that I've got on my mind Ooh, ooh", 'Ooh, ooh Looking for some affirmation', 'Made my way into the sun', 'My friends got their ladies', "And they're all having babies", "I just wanna have some fun I won't bore you with the details, baby", "I don't even wanna waste your time", "Let's just say that maybe", 'You could help me ease my mind', "I ain't Mr. Right But if you're looking for fast love", "If that's love in your eyes", "It's more than enough", "I've had some bad love", "So fast love is all that I've got on my mind Ooh, ooh", 'Baby, baby', 'Ooh, ooh', 'Baby, baby'

In [6]:
len(data)
#there is 2400 lines in this poetry

2400

In [7]:
# Total word number is:
len(" ".join(data))

91330

In [8]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout, Conv1D, MaxPooling1D
from tensorflow.keras.preprocessing.sequence import pad_sequences


#### LSTM model and train test preparation

In [9]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(data) #it's going to fit on the data in the forms of lines.

In [10]:
help(tokenizer)

Help on Tokenizer in module keras.preprocessing.text object:

class Tokenizer(builtins.object)
 |  Tokenizer(num_words=None, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True, split=' ', char_level=False, oov_token=None, analyzer=None, **kwargs)
 |  
 |  Text tokenization utility class.
 |  
 |  Deprecated: `tf.keras.preprocessing.text.Tokenizer` does not operate on
 |  tensors and is not recommended for new code. Prefer
 |  `tf.keras.layers.TextVectorization` which provides equivalent functionality
 |  through a layer which accepts `tf.Tensor` input. See the
 |  [text loading tutorial](https://www.tensorflow.org/tutorials/load_data/text)
 |  for an overview of the layer and text handling in tensorflow.
 |  
 |  This class allows to vectorize a text corpus, by turning each
 |  text into either a sequence of integers (each integer being the index
 |  of a token in a dictionary) or into a vector where the coefficient
 |  for each token could be binary, based on word count, based

In [11]:
encoded_text=tokenizer.texts_to_sequences(data)
encoded_text
# These number actually comes from vocabulary
# it assigns number to each words

[[254, 21, 219, 725],
 [117, 8, 80, 153, 3, 133],
 [14, 10, 726, 727],
 [41, 56, 2, 603, 3, 728, 1, 68, 517, 2, 40, 3, 518, 41],
 [1, 23, 107, 189, 300, 9, 57],
 [286, 35, 46, 10, 230],
 [2, 83, 134, 4, 519, 8, 120],
 [1, 37, 520, 102, 19, 27, 25, 254, 21, 328, 11],
 [27, 209, 11, 13, 9, 124],
 [42, 67, 210, 125],
 [49, 219, 446, 11],
 [22, 328, 11, 28, 14, 10, 45, 220, 34, 8, 120, 53, 53],
 [53, 53, 254, 21, 219, 729],
 [117, 8, 80, 153, 3, 287],
 [8, 447, 220, 521, 730],
 [7, 329, 14, 522, 731],
 [1, 35, 189, 72, 219, 523, 1, 68, 517, 2, 40, 3, 518, 41],
 [1, 23, 107, 189, 300, 9, 57],
 [286, 35, 46, 10, 230],
 [2, 83, 134, 4, 519, 8, 120],
 [1, 37, 520, 102, 19, 27, 25, 254, 21, 328, 11],
 [27, 209, 11, 13, 9, 124],
 [42, 67, 210, 125],
 [45, 49, 219, 446, 11],
 [22, 328, 11, 28, 14, 10, 45, 220, 34, 8, 120, 53, 53],
 [41, 41],
 [53, 53],
 [41, 41],
 [53, 53],
 [41, 41],
 [53, 13, 3, 732, 12, 448],
 [1, 117, 8, 80, 153, 3, 133],
 [390, 733, 391, 34, 198, 4],
 [7, 1, 61, 108, 13, 288

In [12]:
tokenizer.word_counts
# word frequency

OrderedDict([('looking', 12),
             ('for', 145),
             ('some', 15),
             ('education', 2),
             ('made', 30),
             ('my', 370),
             ('way', 43),
             ('into', 23),
             ('the', 632),
             ('night', 26),
             ('all', 174),
             ('that', 255),
             ('bullshit', 2),
             ('conversation', 2),
             ('baby', 86),
             ("can't", 63),
             ('you', 866),
             ('read', 3),
             ('signs', 2),
             ('i', 935),
             ("won't", 51),
             ('bore', 4),
             ('with', 87),
             ('details', 4),
             ("don't", 134),
             ('even', 32),
             ('wanna', 18),
             ('waste', 9),
             ('your', 274),
             ('time', 63),
             ("let's", 10),
             ('just', 100),
             ('say', 74),
             ('maybe', 14),
             ('could', 42),
             ('help', 26),
    

In [13]:
tokenizer.word_index

{'i': 1,
 'you': 2,
 'the': 3,
 'me': 4,
 'to': 5,
 'it': 6,
 'and': 7,
 'my': 8,
 'your': 9,
 'that': 10,
 'love': 11,
 'of': 12,
 'in': 13,
 'all': 14,
 'we': 15,
 'a': 16,
 "i'm": 17,
 'when': 18,
 'but': 19,
 'be': 20,
 'for': 21,
 'so': 22,
 "don't": 23,
 'never': 24,
 "you're": 25,
 'let': 26,
 'if': 27,
 'is': 28,
 'know': 29,
 'up': 30,
 'go': 31,
 'down': 32,
 'will': 33,
 'on': 34,
 'just': 35,
 'do': 36,
 "ain't": 37,
 'heart': 38,
 'like': 39,
 'with': 40,
 'baby': 41,
 "it's": 42,
 'want': 43,
 'this': 44,
 "i've": 45,
 'say': 46,
 'no': 47,
 'as': 48,
 'had': 49,
 'make': 50,
 'can': 51,
 'out': 52,
 'ooh': 53,
 'was': 54,
 'one': 55,
 "can't": 56,
 'time': 57,
 'what': 58,
 'are': 59,
 'feel': 60,
 'see': 61,
 "i'll": 62,
 'fall': 63,
 'there': 64,
 'were': 65,
 'gonna': 66,
 'more': 67,
 "won't": 68,
 'oh': 69,
 'back': 70,
 'face': 71,
 'have': 72,
 'yeah': 73,
 'at': 74,
 "'cause": 75,
 'only': 76,
 'been': 77,
 'stand': 78,
 'ready': 79,
 'way': 80,
 'take': 81,
 'ha

In [14]:
print(f"Number of unique words and total vocab size: {len(tokenizer.word_counts)+1}")
vocab_size=len(tokenizer.word_counts)+1 # always adding plus one for tensorflow

Number of unique words and total vocab size: 1396


In [15]:
x=["play this song"]

In [16]:
tokenizer.texts_to_sequences(x)

[[241, 44, 409]]

### Prepare data for training

In [17]:
 encoded_text

[[254, 21, 219, 725],
 [117, 8, 80, 153, 3, 133],
 [14, 10, 726, 727],
 [41, 56, 2, 603, 3, 728, 1, 68, 517, 2, 40, 3, 518, 41],
 [1, 23, 107, 189, 300, 9, 57],
 [286, 35, 46, 10, 230],
 [2, 83, 134, 4, 519, 8, 120],
 [1, 37, 520, 102, 19, 27, 25, 254, 21, 328, 11],
 [27, 209, 11, 13, 9, 124],
 [42, 67, 210, 125],
 [49, 219, 446, 11],
 [22, 328, 11, 28, 14, 10, 45, 220, 34, 8, 120, 53, 53],
 [53, 53, 254, 21, 219, 729],
 [117, 8, 80, 153, 3, 287],
 [8, 447, 220, 521, 730],
 [7, 329, 14, 522, 731],
 [1, 35, 189, 72, 219, 523, 1, 68, 517, 2, 40, 3, 518, 41],
 [1, 23, 107, 189, 300, 9, 57],
 [286, 35, 46, 10, 230],
 [2, 83, 134, 4, 519, 8, 120],
 [1, 37, 520, 102, 19, 27, 25, 254, 21, 328, 11],
 [27, 209, 11, 13, 9, 124],
 [42, 67, 210, 125],
 [45, 49, 219, 446, 11],
 [22, 328, 11, 28, 14, 10, 45, 220, 34, 8, 120, 53, 53],
 [41, 41],
 [53, 53],
 [41, 41],
 [53, 53],
 [41, 41],
 [53, 13, 3, 732, 12, 448],
 [1, 117, 8, 80, 153, 3, 133],
 [390, 733, 391, 34, 198, 4],
 [7, 1, 61, 108, 13, 288

In [18]:
data_list=[]
for i in encoded_text:
    if len(i)>1:
        for j in range(2,len(i)):
            data_list.append(i[:j])
            print(i[:j])

[254, 21]
[254, 21, 219]
[117, 8]
[117, 8, 80]
[117, 8, 80, 153]
[117, 8, 80, 153, 3]
[14, 10]
[14, 10, 726]
[41, 56]
[41, 56, 2]
[41, 56, 2, 603]
[41, 56, 2, 603, 3]
[41, 56, 2, 603, 3, 728]
[41, 56, 2, 603, 3, 728, 1]
[41, 56, 2, 603, 3, 728, 1, 68]
[41, 56, 2, 603, 3, 728, 1, 68, 517]
[41, 56, 2, 603, 3, 728, 1, 68, 517, 2]
[41, 56, 2, 603, 3, 728, 1, 68, 517, 2, 40]
[41, 56, 2, 603, 3, 728, 1, 68, 517, 2, 40, 3]
[41, 56, 2, 603, 3, 728, 1, 68, 517, 2, 40, 3, 518]
[1, 23]
[1, 23, 107]
[1, 23, 107, 189]
[1, 23, 107, 189, 300]
[1, 23, 107, 189, 300, 9]
[286, 35]
[286, 35, 46]
[286, 35, 46, 10]
[2, 83]
[2, 83, 134]
[2, 83, 134, 4]
[2, 83, 134, 4, 519]
[2, 83, 134, 4, 519, 8]
[1, 37]
[1, 37, 520]
[1, 37, 520, 102]
[1, 37, 520, 102, 19]
[1, 37, 520, 102, 19, 27]
[1, 37, 520, 102, 19, 27, 25]
[1, 37, 520, 102, 19, 27, 25, 254]
[1, 37, 520, 102, 19, 27, 25, 254, 21]
[1, 37, 520, 102, 19, 27, 25, 254, 21, 328]
[27, 209]
[27, 209, 11]
[27, 209, 11, 13]
[27, 209, 11, 13, 9]
[42, 67]
[42, 67, 

#### Paddding

In [19]:
max_length=20
#max length of line is 20 token per line in our poetry

In [20]:
sequences=pad_sequences(data_list,maxlen=max_length,padding="pre") # we set the lenght size equal to 20
sequences

array([[  0,   0,   0, ...,   0, 254,  21],
       [  0,   0,   0, ..., 254,  21, 219],
       [  0,   0,   0, ...,   0, 117,   8],
       ...,
       [  0,   0,   0, ...,  17, 198,  17],
       [  0,   0,   0, ..., 198,  17, 198],
       [  0,   0,   0, ...,  17, 198,   6]])

In [21]:
sequences.shape

(14231, 20)

In [22]:
X=sequences[:,:-1]
y=sequences[:,-1]
print("X values")
print(X)
print("-"*30)
print("X values")
print(y)

X values
[[  0   0   0 ...   0   0 254]
 [  0   0   0 ...   0 254  21]
 [  0   0   0 ...   0   0 117]
 ...
 [  0   0   0 ... 198  17 198]
 [  0   0   0 ...  17 198  17]
 [  0   0   0 ... 198  17 198]]
------------------------------
X values
[ 21 219   8 ...  17 198   6]


In [23]:
X.shape,y.shape

((14231, 19), (14231,))

In [24]:
y=to_categorical(y,num_classes=vocab_size)
#since unique word number is vocab_size, thus there is vocab_size classes
print(y)
print("Shape of y",y.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Shape of y (14231, 1396)


In [25]:
print("Shape of X",X.shape)
seq_length=X.shape[1]
seq_length

Shape of X (14231, 19)


19

#### Build Model
- We will build a simple LSTM model

In [26]:
model=Sequential()
model.add(Embedding(vocab_size,50,input_length=seq_length)) 
#The first layer is the Embedded layer that uses 50-length vectors
#return_sequences=True because we add another LSTM
model.add(LSTM(100,return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100,activation="relu"))
model.add(Dense(vocab_size,activation="softmax")) # we use softmax because there is multiclasses

In [27]:
# help(Embedding)

#### You can use dropout() , You will see that Training will be slightly slower trend in convergence, maybe lower final accuracy

In [28]:
# model=Sequential()
# model.add(Embedding(vocab_size,50,input_length=seq_length)) 
# #The first layer is the Embedded layer that uses 50-length vectors
# #return_sequences=True because we add another LSTM
# model.add(Dropout(0.2))
# model.add(LSTM(100,return_sequences=True))
# model.add(Dropout(0.2))
# model.add(LSTM(100))
# model.add(Dropout(0.2))
# model.add(Dense(100,activation="relu"))
# model.add(Dense(vocab_size,activation="softmax")) # we use softmax because there is multiclasses

####  Dropout can be applied to the input and recurrent connections of the memory units with the LSTM precisely and separately.
#### LSTM-specific dropout may have more pronounced effect on the convergence of the network than the layer-wise dropout.

In [29]:
# model=Sequential()
# model.add(Embedding(vocab_size,50,input_length=seq_length)) 
# #The first layer is the Embedded layer that uses 50-length vectors
# #return_sequences=True because we add another LSTM
# model.add(LSTM(100,return_sequences=True,dropout=0.2,recurrent_dropout=0.2))
# model.add(LSTM(100,dropout=0.2,recurrent_dropout=0.2))
# model.add(Dense(100,activation="relu"))
# model.add(Dense(vocab_size,activation="softmax")) # we use softmax because there is multiclasses

#### LSTM and CNN can be used together

In [30]:
# model=Sequential()
# model.add(Embedding(vocab_size,50,input_length=seq_length)) 
# #The first layer is the Embedded layer that uses 50-length vectors
# #return_sequences=True because we add another LSTM
# model.add(Conv1D(filters=32,kernel_size=3, padding="same",activation="relu"))
# model.add(MaxPooling1D(pool_size=2))
# model.add(LSTM(100,return_sequences=True))
# model.add(LSTM(100))
# model.add(Dense(100,activation="relu"))
# model.add(Dense(vocab_size,activation="softmax")) # we use softmax because there is multiclasses

In [31]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 19, 50)            69800     
                                                                 
 lstm (LSTM)                 (None, 19, 100)           60400     
                                                                 
 lstm_1 (LSTM)               (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 100)               10100     
                                                                 
 dense_1 (Dense)             (None, 1396)              140996    
                                                                 
Total params: 361,696
Trainable params: 361,696
Non-trainable params: 0
_________________________________________________________________


In [32]:
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

In [33]:
model.fit(X,y,batch_size=32,epochs=150)

Epoch 1/150
445/445 [==============================] - 17s 29ms/step - loss: 5.6915 - accuracy: 0.0400
Epoch 2/150
445/445 [==============================] - 14s 31ms/step - loss: 5.2304 - accuracy: 0.0522
Epoch 3/150
445/445 [==============================] - 13s 30ms/step - loss: 5.0358 - accuracy: 0.0628
Epoch 4/150
445/445 [==============================] - 14s 32ms/step - loss: 4.8863 - accuracy: 0.0734
Epoch 5/150
445/445 [==============================] - 13s 29ms/step - loss: 4.7719 - accuracy: 0.0828
Epoch 6/150
445/445 [==============================] - 12s 27ms/step - loss: 4.6617 - accuracy: 0.0948
Epoch 7/150
445/445 [==============================] - 13s 28ms/step - loss: 4.5568 - accuracy: 0.0993
Epoch 8/150
445/445 [==============================] - 13s 28ms/step - loss: 4.4429 - accuracy: 0.1119
Epoch 9/150
445/445 [==============================] - 12s 28ms/step - loss: 4.3251 - accuracy: 0.1254
Epoch 10/150
445/445 [==============================] - 13s 29ms/step - l

445/445 [==============================] - 12s 28ms/step - loss: 0.9798 - accuracy: 0.7465
Epoch 80/150
445/445 [==============================] - 13s 28ms/step - loss: 0.9668 - accuracy: 0.7496
Epoch 81/150
445/445 [==============================] - 12s 28ms/step - loss: 0.9541 - accuracy: 0.7516
Epoch 82/150
445/445 [==============================] - 12s 28ms/step - loss: 0.9408 - accuracy: 0.7531
Epoch 83/150
445/445 [==============================] - 12s 28ms/step - loss: 0.9292 - accuracy: 0.7557
Epoch 84/150
445/445 [==============================] - 12s 28ms/step - loss: 0.9143 - accuracy: 0.7576
Epoch 85/150
445/445 [==============================] - 12s 28ms/step - loss: 0.9059 - accuracy: 0.7600
Epoch 86/150
445/445 [==============================] - 12s 28ms/step - loss: 0.8895 - accuracy: 0.7635
Epoch 87/150
445/445 [==============================] - 12s 28ms/step - loss: 0.8974 - accuracy: 0.7623
Epoch 88/150
445/445 [==============================] - 12s 28ms/step - loss:

### Text Generation

In [34]:
text_lenght= 15 # 15 words per line

def generate_text(input_text, no_lines):
    general_text=[]
    for i in range(no_lines):
        text=[]
        for _ in range(text_lenght):
            encoded=tokenizer.texts_to_sequences([input_text])
            encoded=pad_sequences(encoded,maxlen=seq_length,padding="pre")
            y_pred=np.argmax(model.predict(encoded),axis=-1) # it will generate a word index, loop up into dictionary containing word index
            
            predicted_word=""
            for word,index in tokenizer.word_index.items():
                if index==y_pred:
                    predicted_word=word
                    break
                    
            input_text=input_text +' '+ predicted_word
            text.append(predicted_word)
        
        input_text=text[-1]
        text=" ".join(text) # input text will be the last word of first created line
        general_text.append(text)
    
    return general_text

In [35]:
input_text="me"
text_produced=generate_text(input_text,6)
text_produced

1/1 [==============================] - 0s 20ms/step


["by the seams no bribing me to doubt and i'll do do i never knew",
 'believe is a memory i can tell by the way you walk that the air',
 "she gave you things i didn't give to you old friend why are you so",
 'i spend my whole life hiding my heart away away woke up feeling heavy the',
 "last time let it lose hold me in won't out can you prefer the morning",
 "t show emotions don t know just a place in you and you're in my"]

In [36]:
input_text="i want to see you"
text_produced=generate_text(input_text,6)
text_produced

1/1 [==============================] - 0s 21ms/step


['were away its hole you go at the world town steady as she goes so',
 'i spend my whole life hiding my heart away away woke up feeling heavy the',
 "last time let it lose hold me in won't out can you prefer the morning",
 "t show emotions don t know just a place in you and you're in my",
 'god this reminds so give me till that i went on home to my been',
 "that scares me the more eager i will always still but i know that's where"]